In [2]:
import pandas as pd
df = pd.read_csv('/kaggle/input/rydataa/Data.csv', header=0)
df

,Reviews,Sentiments
0,Sai kha ya her kisi kay bus ki bat nhi hai lak...,Positive
1,sahi bt h,Positive
2,"Kya bt hai,",Positive
3,Wah je wah,Positive
4,Are wha kaya bat hai,Positive
...,...,...
34163,very nice bhook lagne lagi biryani ko dakh k,Positive
34164,koi bna dy.........fishh,Neutral
34165,bi kha k tu dkho bht mazy ki hoti h,Positive
34166,Try ki hai but pata nhi kiu I just don't like ...,Negative


In [3]:
df[df['Sentiments'] == 'Neative']

,Reviews,Sentiments
13277,product achi hai but wrong waist size send kar...,Neative


In [4]:
df.loc[df['Sentiments']=='Neative', 'Sentiments'] = 'Negative'

In [5]:
nan_values = df.isna().sum()

# Print the number of NaN values for each column
print("NaN values per column:")
print(nan_values)

NaN values per column:
Reviews       2
Sentiments    9
dtype: int64


In [6]:
df.dropna(subset=['Reviews', 'Sentiments'], inplace=True)

In [7]:
# Mapping labels to integers
label_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
df['Sentiments'] = df['Sentiments'].map(label_mapping)
df

,Reviews,Sentiments
0,Sai kha ya her kisi kay bus ki bat nhi hai lak...,2.0
1,sahi bt h,2.0
2,"Kya bt hai,",2.0
3,Wah je wah,2.0
4,Are wha kaya bat hai,2.0
...,...,...
34163,very nice bhook lagne lagi biryani ko dakh k,2.0
34164,koi bna dy.........fishh,1.0
34165,bi kha k tu dkho bht mazy ki hoti h,2.0
34166,Try ki hai but pata nhi kiu I just don't like ...,0.0


In [8]:
df.dropna(subset=['Reviews', 'Sentiments'], inplace=True)

In [9]:
import pandas as pd
import tensorflow as tf
from transformers import DistilBertTokenizer 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D

# Tokenization using DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Convert the text column to a list of strings
text_list = df['Reviews'].astype(str).tolist()

# Tokenize the text
encoded_texts = tokenizer(text_list, padding=True, truncation=True, return_tensors='tf')

# Convert the 'input_ids' tensor to a NumPy array
X = encoded_texts['input_ids'].numpy()

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, df['Sentiments'], test_size=0.2, random_state=42)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Bidirectional, Dropout
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer

In [21]:



model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.get_vocab()), output_dim=32, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))  # Assuming 3 classes (Negative, Neutral, Positive)

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 512, 32)           976704    
                                                                 
 spatial_dropout1d_5 (Spati  (None, 512, 32)           0         
 alDropout1D)                                                    
                                                                 
 bidirectional_6 (Bidirecti  (None, 512, 256)          164864    
 onal)                                                           
                                                                 
 bidirectional_7 (Bidirecti  (None, 128)               164352    
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                      

In [22]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

214/214 [==============================] - 10s 45ms/step - loss: 2.5170 - accuracy: 0.5951
Test Accuracy: 59.51%
